# PyOptWrapper

This notebook contains some examples on how to use pyoptwrapper.  First go here: https://github.com/BYUFLOWLab/pyoptsparsewrapper for details on installation and a description of inputs/outputs.

This wrapper makes it easier to use pyoptsparse for our problems.  You can use any of the optimizers defined in pyoptsparse.

In [12]:
from pyoptwrapper import optimize
from pyoptsparse import NSGA2, SNOPT, NLPQLP, ALPSO, SLSQP
import numpy as np

## Unconstrained, gradients provided

In [2]:
# ---- Rosenbrock with gradients, test different optimizers ------


def rosen(x):

    f = (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2
    c = []

    gf = [2*(1 - x[0])*-1 + 200*(x[1] - x[0]**2)*-2*x[0], 200*(x[1] - x[0]**2)]
    gc = []

    return f, c, gf, gc


x0 = [4.0, 4.0]
ub = [5.0, 5.0]
lb = [-5.0, -5.0]


In [3]:
optimizer = NSGA2()
optimizer.setOption('maxGen', 200)

xopt, fopt, info = optimize(rosen, x0, lb, ub, optimizer)
print 'NSGA2:', xopt, fopt, info


optimizer = SNOPT()

xopt, fopt, info = optimize(rosen, x0, lb, ub, optimizer)
print 'SNOPT:', xopt, fopt, info


optimizer = NLPQLP()

xopt, fopt, info = optimize(rosen, x0, lb, ub, optimizer)
print 'NLPQLP:', xopt, fopt, info


optimizer = SLSQP()

xopt, fopt, info = optimize(rosen, x0, lb, ub, optimizer)
print 'SLSQP:', xopt, fopt, info


optimizer = ALPSO()

xopt, fopt, info = optimize(rosen, x0, lb, ub, optimizer)
print 'ALPSO:', xopt, fopt, info

NSGA2: [ 0.9558299   0.91361147] 0.00195099739493 {'fcalls': 19316, 'time': 4.447205066680908}
SNOPT: [ 1.  1.] 1.82925621364e-18 {'code': {'text': 'optimality conditions satisfied', 'value': array([1], dtype=int32)}, 'fcalls': 42, 'time': 0.026717185974121094}
NLPQLP: [ 0.9998208   0.99964098] 3.21551663371e-08 {'fcalls': 46, 'time': 0.01593780517578125}
SLSQP: [ 0.99990575  0.99978891] 5.99147933121e-08 {'fcalls': 48, 'time': 0.02547287940979004}
ALPSO: [ 0.99483871  0.99023567] 5.48999514639e-05 {'fcalls': 1721, 'time': 0.33341193199157715}


# Unconstrained, no graidents

The wrapper automatically detects if gradients are provided or not so you don't need to do anything special.

In [4]:
# ---- Rosenbrock with no gradients ------

def rosen(x):

    f = (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2
    c = []

    return f, c



optimizer = SNOPT()

xopt, fopt, info = optimize(rosen, x0, lb, ub, optimizer)
print 'SNOPT:', xopt, fopt, info

SNOPT: [ 0.99969953  0.99939864] 9.03093498134e-08 {'code': {'text': 'optimality conditions satisfied', 'value': array([1], dtype=int32)}, 'fcalls': 122, 'time': 0.036520957946777344}


# Linear constraints

In [5]:
# ---- test linear constraints ------


H = np.array([[1.0, -1.0], [-1.0, 2.0]])
f = np.array([-2.0, -6.0])
A = np.array([[1.0, 1.0], [-1.0, 2.0], [2.0, 1.0]])
b = np.array([2.0, 2.0, 3.0])


def obj(x):
    ff = 0.5*np.dot(x, np.dot(H, x)) + np.dot(f, x)
    c = []

    return ff, c

x0 = [4.0, 4.0]
ub = [100.0, 100.0]
lb = [0.0, 0.0]


optimizer = SNOPT()

xopt, fopt, info = optimize(obj, x0, lb, ub, optimizer, A, b)
print 'linear constraints:', xopt, fopt, info


linear constraints: [ 0.66666667  1.33333333] -8.22222222222 {'code': {'text': 'optimality conditions satisfied', 'value': array([1], dtype=int32)}, 'fcalls': 8, 'time': 0.0038940906524658203}


# Constrained problem with gradients

In [8]:
# ------ constrained problem ---------

from math import exp

def barnes(x):

    a1 = 75.196
    a3 = 0.12694
    a5 = 1.0345e-5
    a7 = 0.030234
    a9 = 3.5256e-5
    a11 = 0.25645
    a13 = 1.3514e-5
    a15 = -5.2375e-6
    a17 = 7.0e-10
    a19 = -1.6638e-6
    a21 = 0.0005
    a2 = -3.8112
    a4 = -2.0567e-3
    a6 = -6.8306
    a8 = -1.28134e-3
    a10 = -2.266e-7
    a12 = -3.4604e-3
    a14 = -28.106
    a16 = -6.3e-8
    a18 = 3.4054e-4
    a20 = -2.8673

    x1 = x[0]
    x2 = x[1]
    y1 = x1*x2
    y2 = y1*x1
    y3 = x2**2
    y4 = x1**2

    # --- function value ---

    f = a1 + a2*x1 + a3*y4 + a4*y4*x1 + a5*y4**2 + \
        a6*x2 + a7*y1 + a8*x1*y1 + a9*y1*y4 + a10*y2*y4 + \
        a11*y3 + a12*x2*y3 + a13*y3**2 + a14/(x2+1) + \
        a15*y3*y4 + a16*y1*y4*x2 + a17*y1*y3*y4 + a18*x1*y3 + \
        a19*y1*y3 + a20*exp(a21*y1)

    # --- constraints ---

    c = np.zeros(3)
    c[0] = 1 - y1/700.0
    c[1] = y4/25.0**2 - x2/5.0
    c[2] = (x1/500.0- 0.11) - (x2/50.0-1)**2


    # --- derivatives of f ---

    dy1 = x2
    dy2 = y1 + x1*dy1
    dy4 = 2*x1
    dfdx1 = a2 + a3*dy4 + a4*y4 + a4*x1*dy4 + a5*2*y4*dy4 + \
        a7*dy1 + a8*y1 + a8*x1*dy1 + a9*y1*dy4 + a9*y4*dy1 + a10*y2*dy4 + a10*y4*dy2 + \
        a15*y3*dy4 + a16*x2*y1*dy4 + a16*x2*y4*dy1 + a17*y3*y1*dy4 + a17*y3*y4*dy1 + a18*y3 + \
        a19*y3*dy1 + a20*exp(a21*y1)*a21*dy1

    dy1 = x1
    dy2 = x1*dy1
    dy3 = 2*x2
    dfdx2 = a6 + a7*dy1 + a8*x1*dy1 + a9*y4*dy1 + a10*y4*dy2 + \
        a11*dy3 + a12*x2*dy3 + a12*y3 + a13*2*y3*dy3 + a14*-1/(x2+1)**2 + \
        a15*y4*dy3 + a16*y4*y1 + a16*y4*x2*dy1 + a17*y4*y1*dy3 + a17*y4*y3*dy1 + a18*x1*dy3 + \
        a19*y3*dy1 + a19*y1*dy3 + a20*exp(a21*y1)*a21*dy1

    dfdx = np.array([dfdx1, dfdx2])


    # --- derivatives of c ---

    dcdx = np.zeros((3, 2))
    dcdx[0, 0] = -x2/700.0
    dcdx[0, 1] = -x1/700.0
    dcdx[1, 0] = 2*x1/25**2
    dcdx[1, 1] = -1.0/5
    dcdx[2, 0] = 1.0/500
    dcdx[2, 1] = -2*(x2/50.0-1)/50.0

    # dcdx = np.transpose(dcdx)  # matlab format

    return f/30.0, c, dfdx/30.0, dcdx

In [9]:
x0 = np.array([10.0, 10.0])
lb = np.array([0.0, 0.0])
ub = np.array([65.0, 70.0])

optimizer = SNOPT()
xopt, fopt, info = optimize(barnes, x0, lb, ub, optimizer)
print 'barnes:', xopt, fopt, info

barnes: [ 49.52545372  19.62216397] -1.05456035947 {'code': {'text': 'optimality conditions satisfied', 'value': array([1], dtype=int32)}, 'max-c-vio': 1.1140895539440976e-07, 'fcalls': 40, 'time': 0.044798851013183594}


# Multiobjective GA (with a constraint just for fun)

Note that the constarint could actually be a bound constraint.  It's just there to show that NSGA2 can handle constraints.  Because this is a multiobjective problem, xopt contains the entire pareto front, and fopt the objective functions along that front.  

In [11]:
# --------- multiobjective ----------

def multiobj(x):

    f = np.zeros(2)
    f[0] = (x[0]+2)**2 + (x[1]+2)**2 - 10
    f[1] = (x[0]-2)**2 + (x[1]-2)**2 + 20

    c = [x[0] - 1]

    return f, c

x0 = [4.0, 4.0]
ub = [20.0, 20.0]
lb = [-20.0, -20.0]

optimizer = NSGA2()
optimizer.setOption('maxGen', 200)

xopt, fopt, info = optimize(multiobj, x0, lb, ub, optimizer)
print 'NSGA2:'
print 'xopt =', xopt
print 'fopt =', fopt
print 'info =', info


NSGA2:
xopt = [[ 0.9973342   2.009444  ]
 [-2.000297   -1.999851  ]
 [-0.8424979  -0.9744967 ]
 [ 0.02680356  0.03089153]
 [-0.5519721  -0.4703299 ]
 [ 0.2442713   0.08768983]
 [-1.153866   -1.099569  ]
 [ 0.9913403   0.8927175 ]
 [-1.138731   -0.9416537 ]
 [-0.9940932  -0.9199558 ]
 [-0.5967224  -1.017377  ]
 [-0.4262689  -0.793217  ]
 [-0.5967224  -1.017377  ]
 [ 0.01214987 -0.07410228]
 [ 0.99112     0.6686376 ]
 [ 0.9948042   0.8117054 ]
 [-1.654517   -1.423697  ]
 [ 0.414778   -0.1967281 ]
 [-1.282795   -1.533888  ]
 [-1.614406   -1.540971  ]
 [ 0.9948919   1.946815  ]
 [ 0.7902784   0.932416  ]
 [ 0.6830573   0.4643452 ]
 [ 0.9548193   0.5935744 ]
 [-0.1937757  -0.05834315]
 [-0.5519721  -0.8497754 ]
 [ 0.9948919   1.47879   ]
 [-1.303152   -1.610229  ]
 [ 0.6830573   0.6480824 ]
 [ 0.4844151   0.3555492 ]
 [ 0.9811146   1.135902  ]
 [ 0.9913403   1.015234  ]
 [-1.591907   -1.994926  ]
 [ 0.9972808   1.369168  ]
 [-1.099699   -1.20393   ]
 [ 0.9973342   1.786137  ]
 [ 0.1110038  